# **Data Loader de Grafos – Flujo de Trabajo Unificado**

## **OBJETIVO:**
Transformar grafos crudos en una representación uniforme, limpia y explotable.

In [ ]:
%run librerias.ipynb

## **1. CARGA DEL GRAFO CRUDO**

In [ ]:
def cargar_grafo_crudo(n=50, p=0.05):
    """
    Simula la carga de un grafo crudo (estilo SNAP/DIMACS).
    - Nodos no consecutivos
    - Posibles lazos y duplicados
    """
    G = nx.MultiGraph()
    for i in range(n):
        G.add_node(i * random.randint(1, 3))

    nodos = list(G.nodes())
    for u in nodos:
        for v in nodos:
            if random.random() < p:
                G.add_edge(u, v)
    return G

## **2. NORMALIZACIÓN DE LA ESTRUCTURA**

In [ ]:
def normalizar_estructura(G):
    """
    - Convierte a grafo simple
    - Elimina lazos
    - Elimina aristas duplicadas
    """
    H = nx.Graph()
    for u, v in G.edges():
        if u != v:
            H.add_edge(u, v)
    return H

## **3. REINDEXACIÓN DE VÉRTICES**

In [ ]:
def reindexar_vertices(G):
    """
    Mapea nodos a índices consecutivos {0,1,...,n-1}
    """
    mapping = {node: i for i, node in enumerate(G.nodes())}
    return nx.relabel_nodes(G, mapping)

## **4. VERIFICACIÓN DE PROPIEDADES BÁSICAS**

In [ ]:
def propiedades_basicas(G):
    n = G.number_of_nodes()
    m = G.number_of_edges()
    densidad = nx.density(G)
    grado_max = max(dict(G.degree()).values()) if n > 0 else 0
    return n, m, densidad, grado_max

## **5. EXTRACCIÓN DE FEATURES ESTRUCTURALES**

In [ ]:
def extraer_features(G):
    """
    Features por nodo:
    - Grado
    - Coeficiente de clustering
    - Número de core (k-core)
    """
    grados = dict(G.degree())
    clustering = nx.clustering(G)
    core = nx.core_number(G) if G.number_of_edges() > 0 else {v: 0 for v in G.nodes()}

    X = []
    for v in G.nodes():
        X.append([grados[v], clustering[v], core[v]])
    return np.array(X)

## **6. NORMALIZACIÓN DE FEATURES**

In [ ]:
def normalizar_features(X):
    """
    Normalización: media 0, varianza 1
    """
    mean = X.mean(axis=0)
    std = X.std(axis=0) + 1e-8
    return (X - mean) / std

## **7. REPRESENTACIÓN COMPUTACIONAL**

In [ ]:
def representacion_computable(G, X):
    """
    - Edge list indexada
    - Matriz de features
    """
    edge_index = np.array(list(G.edges())).T
    return edge_index, X

## **8. DATA LOADER COMPLETO**

In [ ]:
def data_loader():
    # 1. Carga
    G = cargar_grafo_crudo()

    # 2. Normalización
    G = normalizar_estructura(G)

    # 3. Reindexación
    G = reindexar_vertices(G)

    # 4. Estadísticas
    n, m, densidad, grado_max = propiedades_basicas(G)

    # 5. Features
    X = extraer_features(G)

    # 6. Normalización
    X = normalizar_features(X)

    # 7. Representación final
    edge_index, X = representacion_computable(G, X)

    return {
        "grafo": G,
        "edge_index": edge_index,
        "features": X,
        "estadisticas": {
            "nodos": n,
            "aristas": m,
            "densidad": densidad,
            "grado_max": grado_max
        }
    }

## **9. EJECUCIÓN DEL PIPELINE**

In [ ]:
data = data_loader()

print("Estadísticas del grafo:")
print(data["estadisticas"])
print("\nShape de features:", data["features"].shape)
print("Shape de edge_index:", data["edge_index"].shape)

G = data["grafo"]
stats = data["estadisticas"]

print(f"\nGrafo recibido: {stats['nodos']} nodos, {stats['aristas']} aristas")
print(f"Densidad: {stats['densidad']}, Grado máximo: {stats['grado_max']}")